In [309]:
import numpy as np
import copy

In [310]:
### Something that reacts to a signal
class SignalSensitive:
  # Gets triggered when the `at_index`-th
  # input of a component
  # receives a signal
  def on_input_changed(self, at_index, signal):
    raise NotImplementedError()

### Info about an outgoing connection
### The component is connected to the `at_index`-th input
### of `to_component`
class ConnectionInfo:
  # to_index        :: Int
  # to_component    :: Component -- or SignalSensitive
  def __init__(self, to_index, to_component):
    self.to_index = to_index
    self.to_component = to_component

  def notify(self, signal):
    if self.to_component == None:
      return
    self.to_component.on_input_changed(self.to_index,
                                       signal)
    
  def copy(self, already_copied = dict(), talk=False):
    if talk:
      print(f'[ConnectionInfo] Invoking copy on {self.to_component} of type {type(self)}')
    if self == NoConnection or self.to_component is None:
      return NoConnection
    ci = ConnectionInfo(to_index=self.to_index,
                        to_component=self.to_component.copy(already_copied))
    return ci

### Placeholder for empty slots
NoConnection = ConnectionInfo(to_index=-1,
                              to_component=None)

In [311]:
class Component(SignalSensitive):
  # n_input_slots                 :: Int       
  # n_output_slots                :: Int
  # input_slots                   :: List<Signal>
  # output_slots                  :: List<Signal>
  # connections                   :: List<ConnectionInfo>
  def __init__(self,
               n_input_slots,
               n_output_slots,
               name='Unknown'):
    self.n_input_slots = n_input_slots
    self.n_output_slots = n_output_slots
    self.input_slots = [False for _ in range(n_input_slots)]
    self.output_slots = [False for _ in range(n_output_slots)]
    self.connections = [NoConnection for _ in range(n_output_slots)]  
    self.input_connections = [None for _ in range(n_input_slots)]

  def connect_at_index(self, my_output_index, neighbor_input_index, neighbor):
    if self.connections[my_output_index] != NoConnection:
      raise ValueError("This slot is already connected to something")
    
    self.connections[my_output_index] = ConnectionInfo(to_index=neighbor_input_index,
                                                       to_component=neighbor)
    neighbor.input_connections[neighbor_input_index] = self    
    self.connections[my_output_index].notify(self.output_slots[my_output_index])  

  def disconnect_at_index(self, my_output_index):
    ### Sends an empty signal
    self.connections[my_output_index].notify(False)
    self.connections[my_output_index] = NoConnection

  def get_shallow_copy_instance(self):
    raise NotImplementedError("Raised shallow in component")

  def copy(self, already_copied=dict(), talk=False):
    if talk:
      print(f'[Component] Invoking copy on {self} of type {type(self)}')

    my_id = id(self)
    if my_id in already_copied:
      return already_copied[my_id]
    new_obj = self.get_shallow_copy_instance()
    # Assume that input is already a new copy
    new_obj.input_connections = [comp for comp in self.input_connections]
    # Use that assumption for the recursive call
    already_copied[my_id] = new_obj
    new_obj.connections = [conn.copy(already_copied, talk) if conn is not None else None for conn in self.connections]
    return new_obj

  def khash(self):
    return hex(id(self) % 200 + 13*(self.n_input_slots + 7*self.n_output_slots)**2)

  def __str__(self):
    return 'An unknown component'

In [312]:
class Wire(Component):
  def __init__(self,
               orientation='horizontal'):
    super(Wire, self).__init__(n_input_slots=1,
                               n_output_slots=1)
    self.orientation = orientation
  
  def resize(self, n):
    if n == self.n_output_slots:
      return
    if n <= 0:
      n = 1
    if n < self.n_output_slots:
      self.output_slots = self.output_slots[:n]
      ## TODO: Traverse from n-th and send empty signal
    else:
      self.output_slots = self.output_slots + [False for _ in range(n - self.n_output_slots)]
    self.n_output_slots = n
  
  def on_input_changed(self, at_index, signal):
    if self.input_slots[at_index] == signal:
      return
    
    self.input_slots[at_index] = signal
    for conn in self.connections:
      conn.notify(signal)
  
  def serialize(self):
    return self.input_connections[0].serialize()

  def get_shallow_copy_instance(self):
    return Wire(orientation=self.orientation)

  def __str__(self):
    return self.khash() + ' (Wire)'



In [313]:
### Two inputs, one output
class BinaryGate(Component):
  # op :: (Signal, Signal) -> Signal
  def __init__(self, op):
    super(BinaryGate, self).__init__(n_input_slots=2,
                                     n_output_slots=1)
    self.op = op

  def i1(self):
    return self.input_slots[0]

  def i2(self):
    return self.input_slots[1]

  def o(self):
    return self.output_slots[0]

  # wrapper
  def connect_output(self, to_index, to_component):
    self.connect_at_index(0, to_index, to_component)

  def on_input_changed(self, at_index, signal):
    if self.input_slots[at_index] == signal:
      return
    self.input_slots[at_index] = signal
    new_output = self.op(self.i1(), self.i2())
    if new_output == self.o():
      return
    self.output_slots[0] = new_output
    for conn in self.connections:
      conn.notify(new_output)




In [314]:
class AndGate(BinaryGate):
  def __init__(self):
    super(AndGate, self).__init__(op = lambda a, b: a and b)
  def __str__(self):
    return self.khash() + ' (AND Gate)'
  def get_shallow_copy_instance(self):
    return AndGate()

class OrGate(BinaryGate):
  def __init__(self):
    super(OrGate, self).__init__(op = lambda a, b: a or b)
  def __str__(self):
      return self.khash() + ' (OR Gate)'
  def get_shallow_copy_instance(self):
    return OrGate()

class XorGate(BinaryGate):
  def __init__(self):
    super(XorGate, self).__init__(op = lambda a, b: a != b)
  def __str__(self):
      return self.khash() + ' (XOR Gate)'
  def get_shallow_copy_instance(self):
    return XorGate()

In [315]:
class Pin(Component):
  def __init__(self, val=False):
    super(Pin, self).__init__(n_input_slots=0,
                              n_output_slots=1)
    self.output_slots[0] = val
  
  def flip(self):
    self.output_slots[0] = not self.output_slots[0]
    for conn in self.connections:
      conn.notify(self.output_slots[0])
  
  def set_val(self, val):
    if self.output_slots[0] == val:
      return
    self.flip()

  def get_shallow_copy_instance(self):
    return Pin(val=False)

  def __str__(self):
      return self.khash() + ' (Pin)'


In [316]:
class LightBulb(Component):
  def __init__(self):
    super(LightBulb, self).__init__(n_input_slots=1,
                                    n_output_slots=1)
  
  def connect_at_index(self, at_index, to_component):
    raise ValueError("Lightbulb cannot be connected to anything")

  def serialize(self):
    return self.input_connections[0].serialize()

  def on_input_changed(self, at_index, signal):
    self.input_slots[at_index] = signal
    self.output_slots[at_index] = signal

  def get_shallow_copy_instance(self):
    return LightBulb()

  def __str__(self):
      return self.khash() + ' (LightBulb)'


In [317]:
class CompositeComponent(Component):
  def __init__(self, pins, lightbulbs):
    super(CompositeComponent, self).__init__(n_input_slots=len(pins),
                                             n_output_slots=len(lightbulbs))
    
    self.pins = [p.copy() for p in pins]
    self.lightbulbs = [lb.copy() for lb in lightbulbs]
    
  
  def on_input_changed(self, at_index, signal):
    snapshot = [o for o in lb.output_slots]
    self.pins[at_index].on_input_changed(at_index=0,
                                         signal=signal)
    for i, old_signal, new_signal in enumerate(zip(snapshot, self.lightbulbs)):
      if old_signal == new_signal:
        continue
      self.connections[i].notify(new_signal)

  def connect_at_index(self, my_output_index, neighbor_input_index, neighbor):
    pass
      

    


  def get_shallow_copy_instance(self):
    pass

  # @Override
  def copy(self):
    # TODO: Figure this out!
    pass

  def __str__(self):
      return self.khash() + ' (Composite)'

  

In [318]:
A0 = Pin(False)
A0.name = 'A0'
A1 = Pin(False)
A1.name = 'A1'
A2 = Pin(False)
A2.name = "A2"

and1 = AndGate()
xor1 = XorGate()

w1 = Wire()
w1.resize(2)

w2 = Wire()
w2.resize(2)

w3 = Wire()
w3.resize(2)

w4 = Wire()
w4.resize(2)

lb = LightBulb()

A0.connect_at_index(my_output_index=0,
                    neighbor_input_index=0,
                    neighbor=w1)

A1.connect_at_index(my_output_index=0,
                    neighbor_input_index=0,
                    neighbor=w2)

A2.connect_at_index(my_output_index=0,
                    neighbor_input_index=0,
                    neighbor=w3)

w1.connect_at_index(my_output_index=0,
                    neighbor_input_index=0,
                    neighbor=and1)

w2.connect_at_index(my_output_index=0,
                    neighbor_input_index=1,
                    neighbor=and1)

w3.connect_at_index(my_output_index=0,
                    neighbor_input_index=1,
                    neighbor=xor1)

and1.connect_at_index(my_output_index=0,
                      neighbor_input_index=0,
                      neighbor=xor1)

xor1.connect_at_index(my_output_index=0,
                      neighbor_input_index=0,
                      neighbor=w4)

w4.connect_at_index(my_output_index=0,
                    neighbor_input_index=0,
                    neighbor=lb)

print(lb.output_slots)
A0.flip()
A1.flip()
print(lb.output_slots)

[False]
[True]


In [319]:
def traverse_circuit(element, depth=0):
  # idd = id(element)
  # idd = element.identify()
  # idd = hex(id(element))
  print(''.join('\t'*depth),'-->', element)
  for c in element.connections:
    if c is None or c.to_component is None:
      continue
    traverse_circuit(c.to_component, depth+1)

In [320]:
traverse_circuit(A0)
traverse_circuit(A1)
traverse_circuit(A2)

 --> 0x2bd (Pin)
	 --> 0xc05 (Wire)
		 --> 0x495 (AND Gate)
			 --> 0x435 (XOR Gate)
				 --> 0xbcd (Wire)
					 --> 0x368 (LightBulb)
 --> 0x2fd (Pin)
	 --> 0xbd5 (Wire)
		 --> 0x495 (AND Gate)
			 --> 0x435 (XOR Gate)
				 --> 0xbcd (Wire)
					 --> 0x368 (LightBulb)
 --> 0x33d (Pin)
	 --> 0xc0d (Wire)
		 --> 0x435 (XOR Gate)
			 --> 0xbcd (Wire)
				 --> 0x368 (LightBulb)


In [322]:
composite = CompositeComponent(pins=[A0, A1, A2],
                               lightbulbs=[lb])

traverse_circuit(composite.pins[0])
traverse_circuit(composite.pins[1])
traverse_circuit(composite.pins[2])

# print(f'Idevi originalnih pinova: {[id(p) for p in [A0, A1, A2]]}')
# print(f'Idevi novih pinova: {[id(p) for p in composite.pins]}')

 --> 0x315 (Pin)
	 --> 0x368 (Wire)
		 --> 0x4cd (AND Gate)
			 --> 0x475 (XOR Gate)
				 --> 0x3e0 (Wire)
					 --> 0x360 (LightBulb)
 --> 0x315 (Pin)
	 --> 0x3f8 (Wire)
		 --> 0x4cd (AND Gate)
			 --> 0x475 (XOR Gate)
				 --> 0x3e0 (Wire)
					 --> 0x360 (LightBulb)
 --> 0x32d (Pin)
	 --> 0x3f0 (Wire)
		 --> 0x475 (XOR Gate)
			 --> 0x3e0 (Wire)
				 --> 0x360 (LightBulb)


fatal: not a git repository (or any of the parent directories): .git
